In [17]:
library(h2o)
localH2O <- h2o.init(min_mem_size = '10G', nthreads = 4)


H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    /tmp/Rtmp8Xw9Rw/h2o_admin_started_from_r.out
    /tmp/Rtmp8Xw9Rw/h2o_admin_started_from_r.err


..Successfully connected to http://127.0.0.1:54321/ 

R is connected to the H2O cluster: 
    H2O cluster uptime:         1 seconds 643 milliseconds 
    H2O cluster version:        3.6.0.8 
    H2O cluster name:           H2O_started_from_R_admin_klv352 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   9.58 GB 
    H2O cluster total cores:    4 
    H2O cluster allowed cores:  4 
    H2O cluster healthy:        TRUE 



In [18]:
trainset.hex <- h2o.uploadFile(path = 'trainset_values.csv', destination_frame = 'trainset.hex', sep = ',', header = TRUE)
labels.hex <- h2o.uploadFile(path = 'trainset_labels.csv', destination_frame = 'labels.hex', sep = ',', header = TRUE)

trainsetFull.hex <- h2o.merge(trainset.hex, labels.hex)
trainsetFull.split <- h2o.splitFrame(trainsetFull.hex)

allVariables <- colnames(trainsetFull.hex)
predictors <- colnames(trainsetFull.hex)[!(allVariables 
                                           %in% c('id', 'wpt_name', 'subvillage', 'scheme_name', 'installer', 'funder',
                                                  'status_group', 'ward_lga'))]

  |======================================================================| 100%
  |======================================================================| 100%


In [19]:
training <- trainsetFull.split[[1]]
training$merged_status <- h2o.ifelse(training$status_group %in% c('functional', 'functional needs repair'),
                                    'functional', 'non functional')
head(training)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,ellip.h,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,merged_status
1,69572,6000,1.300061e+12,Roman,1390,Roman,34.93809,-9.856322,none,0,⋯,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,functional
2,34310,25,1.36175e+12,Lottery Club,686,World vision,37.46066,-3.821329,Kwa Mahundi,0,⋯,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional,functional
3,67743,0,1.359331e+12,Unicef,263,UNICEF,38.48616,-11.1553,Zahanati Ya Nanyumbu,0,⋯,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional,non functional
4,19728,0,1.310515e+12,Action In A,0,Artisan,31.13085,-1.825359,Shuleni,0,⋯,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional,functional
5,9944,20,1.299974e+12,Mkinga Distric Coun,0,DWE,39.1728,-4.765587,Tajiri,0,⋯,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe,functional,functional
6,19816,0,1.34905e+12,Dwsp,0,DWSP,33.36241,-3.766365,Kwa Ngomho,0,⋯,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,non functional,non functional


In [20]:
table(as.vector(training$merged_status))


    functional non functional 
         27385          17116 

In [22]:
modelRf <- h2o.randomForest(predictors, 'merged_status', training)
summary(modelRf)

Warning message:
In .h2o.validateModelParameters(algo, param_values, h2oRestApiVersion): Dropping constant columns: recorded_by.


  |======================================================================| 100%
Model Details:

H2OBinomialModel: drf
Model Key:  DRF_model_R_1452268832167_5 
Model Summary: 
  number_of_trees model_size_in_bytes min_depth max_depth mean_depth min_leaves
1              50             1962420        20        20   20.00000       2566
  max_leaves mean_leaves
1       3316  3080.76000

H2OBinomialMetrics: drf
** Reported on training data. **
Description: Metrics reported on Out-Of-Bag training samples

MSE:  0.1113366
R^2:  0.5296053
LogLoss:  0.370817
AUC:  0.9141627
Gini:  0.8283254

Confusion Matrix for F1-optimal threshold:
               non functional functional    Error         Rate
non functional          11783       5333 0.311580  =5333/17116
functional               1421      25964 0.051890  =1421/27385
Totals                  13204      31297 0.151772  =6754/44501

Maximum Metrics: Maximum metrics at their respective thresholds
                      metric threshold    value id

In [23]:
testing <- trainsetFull.split[[2]]
testing$merged_status <- h2o.ifelse(testing$status_group %in% c('functional', 'functional needs repair'),
                                    'functional', 'non functional')

h2o.performance(modelRf, testing)

H2OBinomialMetrics: drf

MSE:  0.1078399
R^2:  0.5437038
LogLoss:  0.3499881
AUC:  0.9193597
Gini:  0.8387194

Confusion Matrix for F1-optimal threshold:
               non functional functional    Error         Rate
non functional           4169       1539 0.269622   =1539/5708
functional                582       8609 0.063323    =582/9191
Totals                   4751      10148 0.142359  =2121/14899

Maximum Metrics: Maximum metrics at their respective thresholds
                      metric threshold    value idx
1                     max f1  0.570296 0.890325 215
2                     max f2  0.352007 0.938556 285
3               max f0point5  0.763396 0.879687 134
4               max accuracy  0.570296 0.857641 215
5              max precision  0.997737 0.995086   2
6           max absolute_MCC  0.570296 0.695829 215
7 max min_per_class_accuracy  0.714485 0.834838 155

In [49]:
training.step1 <- h2o.cbind(training, as.factor(predict(modelRf, training)$predict))
head(training.step1)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,ellip.h,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,merged_status,predict
1,69572,6000,1.300061e+12,Roman,1390,Roman,34.93809,-9.856322,none,0,⋯,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,functional,functional
2,34310,25,1.36175e+12,Lottery Club,686,World vision,37.46066,-3.821329,Kwa Mahundi,0,⋯,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional,functional,functional
3,67743,0,1.359331e+12,Unicef,263,UNICEF,38.48616,-11.1553,Zahanati Ya Nanyumbu,0,⋯,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional,non functional,non functional
4,19728,0,1.310515e+12,Action In A,0,Artisan,31.13085,-1.825359,Shuleni,0,⋯,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional,functional,functional
5,9944,20,1.299974e+12,Mkinga Distric Coun,0,DWE,39.1728,-4.765587,Tajiri,0,⋯,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe,functional,functional,functional
6,19816,0,1.34905e+12,Dwsp,0,DWSP,33.36241,-3.766365,Kwa Ngomho,0,⋯,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,non functional,non functional,non functional


In [37]:
training.functionalgroup <- training.step1[training.step1$predict == 'functional',]
training.nonfunctionalgroup <- training.step1[training.step1$predict == 'non functional',]
head(training.functionalgroup)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,ellip.h,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,merged_status,predict,non functional,functional
1,69572,6000,1.300061e+12,Roman,1390,Roman,34.93809,-9.856322,none,0,⋯,spring,spring,groundwater,communal standpipe,communal standpipe,functional,functional,functional,0.002361227,0.9976388
2,34310,25,1.36175e+12,Lottery Club,686,World vision,37.46066,-3.821329,Kwa Mahundi,0,⋯,dam,dam,surface,communal standpipe multiple,communal standpipe,functional,functional,functional,0.0402252,0.9597748
3,19728,0,1.310515e+12,Action In A,0,Artisan,31.13085,-1.825359,Shuleni,0,⋯,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional,functional,functional,0.07403299,0.925967
4,9944,20,1.299974e+12,Mkinga Distric Coun,0,DWE,39.1728,-4.765587,Tajiri,0,⋯,other,other,unknown,communal standpipe multiple,communal standpipe,functional,functional,functional,0.01870357,0.9812964
5,54551,0,1.349741e+12,Rwssp,0,DWE,32.62062,-4.226198,Tushirikiane,0,⋯,shallow well,shallow well,groundwater,hand pump,hand pump,non functional,non functional,functional,0.3795639,0.6204361
6,53934,0,1.351901e+12,Wateraid,0,Water Aid,32.7111,-5.146712,Kwa Ramadhan Musa,0,⋯,machine dbh,borehole,groundwater,hand pump,hand pump,non functional,non functional,functional,0.4267854,0.5732146


In [39]:
rfModel.functional.step2 <- h2o.randomForest(predictors, 'status_group', training.functionalgroup)
summary(rfModel.functional.step2)

Warning message:
In .h2o.validateModelParameters(algo, param_values, h2oRestApiVersion): Dropping constant columns: recorded_by.


  |======================================================================| 100%
Model Details:

H2OMultinomialModel: drf
Model Key:  DRF_model_R_1452268832167_35 
Model Summary: 
  number_of_trees model_size_in_bytes min_depth max_depth mean_depth min_leaves
1             150             4107835        20        20   20.00000       1430
  max_leaves mean_leaves
1       3273  2147.62670

H2OMultinomialMetrics: drf
** Reported on training data. **
Description: Metrics reported on Out-Of-Bag training samples

Training Set Metrics: 
Metrics reported on Out-Of-Bag training samples 

Extract training frame with `h2o.getFrame("RTMP_38")`
MSE: (Extract with `h2o.mse`) 0.1691251
R^2: (Extract with `h2o.r2`) 0.6310181
Logloss: (Extract with `h2o.logloss`) 0.7048893
Confusion Matrix: Extract with `h2o.confusionMatrix(<model>,train = TRUE)`)
                        functional functional.needs.repair non.functional
functional                   23041                     454            344
functional

In [50]:
testing.step1 <- h2o.cbind(testing, as.factor(predict(modelRf, testing)$predict))
testing.functionalgroup <- testing.step1[testing.step1$predict == 'functional',]
testing.nonfunctionalgroup <- testing.step1[testing.step1$predict == 'non functional',]
head(testing.functionalgroup)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,ellip.h,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,merged_status,predict
1,8776,0,1.362528e+12,Grumeti,1399,GRUMETI,34.69877,-2.147466,Zahanati,0,⋯,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional,functional,functional
2,50409,200,1.361146e+12,Danida,1062,DANIDA,35.77026,-10.57417,Kwa Alid Nchimbi,0,⋯,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional,functional,functional
3,36957,0,1.350173e+12,World Vision,0,World vision,33.79811,-3.290194,Pamba,0,⋯,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional,functional,functional
4,34169,0,1.311293e+12,Hesawa,1162,DWE,32.92015,-1.947868,Ngomee,0,⋯,insufficient,insufficient,spring,spring,groundwater,other,other,functional needs repair,functional,functional
5,18274,500,1.298333e+12,Danida,1763,Danid,34.50897,-9.894412,none,0,⋯,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,functional,functional
6,41583,0,1.298419e+12,NA,-41,NA,39.81291,-7.889986,Msikitini Wa Ijumaa,0,⋯,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional,functional,functional


In [51]:
h2o.performance(rfModel.step2, testing.functionalgroup)

H2OMultinomialMetrics: drf

Test Set Metrics: 

MSE: (Extract with `h2o.mse`) 0.1885404
R^2: (Extract with `h2o.r2`) 0.6690639
Logloss: (Extract with `h2o.logloss`) 0.6323026
Confusion Matrix: Extract with `h2o.confusionMatrix(<model>, <data>)`)
                        functional functional.needs.repair non.functional
functional                    7526                     117             71
functional needs repair        632                     349             20
non functional                1507                      86            103
Totals                        9665                     552            194
                             Error           Rate
functional              0.02437127    188 / 7,714
functional needs repair 0.65134865    652 / 1,001
non functional          0.93926887  1,593 / 1,696
Totals                  0.23369513 2,433 / 10,411

Hit Ratio Table: Extract with `h2o.hit_ratio_table(<model>, <data>)`
Top-3 Hit Ratios: 
  k hit_ratio
1 1  0.766305
2 2  0.942176
3 3

In [52]:
colnames(testing.functionalgroup)
ncol(testing.functionalgroup)
colnames(testing.nonfunctionalgroup)
ncol(testing.nonfunctionalgroup)

[1] "id"                    "amount_tsh"            "date_recorded"        
 [4] "funder"                "gps_height"            "installer"            
 [7] "longitude"             "latitude"              "wpt_name"             
[10] "num_private"           "basin"                 "subvillage"           
[13] "region"                "region_code"           "district_code"        
[16] "lga"                   "ward"                  "population"           
[19] "public_meeting"        "recorded_by"           "scheme_management"    
[22] "scheme_name"           "permit"                "construction_year"    
[25] "extraction_type"       "extraction_type_group" "extraction_type_class"
[28] "management"            "management_group"      "payment"              
[31] "payment_type"          "water_quality"         "quality_group"        
[34] "quantity"              "quantity_group"        "source"               
[37] "source_type"           "source_class"          "waterpoint_type"      
[40] "waterpoint_type_group" "status_group"          "merged_status"        
[43] "predict"

[1] 43

[1] "id"                    "amount_tsh"            "date_recorded"        
 [4] "funder"                "gps_height"            "installer"            
 [7] "longitude"             "latitude"              "wpt_name"             
[10] "num_private"           "basin"                 "subvillage"           
[13] "region"                "region_code"           "district_code"        
[16] "lga"                   "ward"                  "population"           
[19] "public_meeting"        "recorded_by"           "scheme_management"    
[22] "scheme_name"           "permit"                "construction_year"    
[25] "extraction_type"       "extraction_type_group" "extraction_type_class"
[28] "management"            "management_group"      "payment"              
[31] "payment_type"          "water_quality"         "quality_group"        
[34] "quantity"              "quantity_group"        "source"               
[37] "source_type"           "source_class"          "waterpoint_type"      
[40] "waterpoint_type_group" "status_group"          "merged_status"        
[43] "predict"

[1] 43

In [56]:
library(magrittr)
testings.preds <- testing.functionalgroup[,-43] %>% 
    h2o.cbind(., as.factor(predict(rfModel.functional.step2, testing.functionalgroup)$predict)) %>%
    h2o.rbind(., testing.nonfunctionalgroup)
head(testings.preds)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,ellip.h,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,merged_status,predict
1,8776,0,1.362528e+12,Grumeti,1399,GRUMETI,34.69877,-2.147466,Zahanati,0,⋯,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional,functional,functional
2,50409,200,1.361146e+12,Danida,1062,DANIDA,35.77026,-10.57417,Kwa Alid Nchimbi,0,⋯,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional,functional,functional
3,36957,0,1.350173e+12,World Vision,0,World vision,33.79811,-3.290194,Pamba,0,⋯,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional,functional,functional
4,34169,0,1.311293e+12,Hesawa,1162,DWE,32.92015,-1.947868,Ngomee,0,⋯,insufficient,insufficient,spring,spring,groundwater,other,other,functional needs repair,functional,functional needs repair
5,18274,500,1.298333e+12,Danida,1763,Danid,34.50897,-9.894412,none,0,⋯,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,functional,functional
6,41583,0,1.298419e+12,NA,-41,NA,39.81291,-7.889986,Msikitini Wa Ijumaa,0,⋯,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional,functional,functional needs repair


In [59]:
cm <- table(as.vector(testings.preds$status_group), as.vector(testings.preds$predict))
cm

                         
                          functional functional needs repair non functional
  functional                    7501                     141            450
  functional needs repair        635                     343            121
  non functional                1501                      89           4118

In [61]:
sum(diag(cm)) / sum(cm)
cm[2,2] / sum(cm[2,])

[1] 0.8028727

[1] 0.3121019

Shutdown cluster:

In [16]:
h2o.shutdown(prompt = FALSE)